<a href="https://colab.research.google.com/github/GyanLevy/Cloud_Computing/blob/main/Tirgul2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
print("Dictionary: ", json.dumps({"name": "John", "age": 30}))
print("List:       ", json.dumps(["apple", "bananas"]))
print("Tuple:      ", json.dumps(("apple", "bananas")))
print("String:     ", json.dumps("hello"))
print("Integer:    ", json.dumps(42))
print("Float:      ", json.dumps(31.76))
print("True:       ", json.dumps(True))
print("False:      ", json.dumps(False))
print("None:       ", json.dumps(None))

Dictionary:  {"name": "John", "age": 30}
List:        ["apple", "bananas"]
Tuple:       ["apple", "bananas"]
String:      "hello"
Integer:     42
Float:       31.76
True:        true
False:       false
None:        null


In [2]:
import json
json_data = '{"acronym": "BLD", "name": "Boulder Colorado", \
              "latitude": 40.00, "longitude": -105.25}'
python_obj = json.loads(json_data)
print("Name:     ", python_obj["name"])
print("Acronym:  ", python_obj["acronym"])
print("Latitude: ", python_obj["latitude"])
print("Latitude: ", python_obj["longitude"])

print()
print(json.dumps(python_obj, sort_keys=True, indent=4))

Name:      Boulder Colorado
Acronym:   BLD
Latitude:  40.0
Latitude:  -105.25

{
    "acronym": "BLD",
    "latitude": 40.0,
    "longitude": -105.25,
    "name": "Boulder Colorado"
}


In [6]:
import json
from google.colab import drive
drive.mount('/content/drive')

x = {
  "name": "John",
  "age": 30,
  "married": True,
  "divorced": False,
  "children": ("Ann","Billy"),
  "pets": None,
  "cars": [
    {"model": "BMW 230", "mpg": 27.5},
    {"model": "Ford Edge", "mpg": 24.1}
  ]
}

with open('/content/drive/My Drive/Colab Notebooks/s1.json', "w") as fid:
     json.dump(x, fid)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
# -*- coding: utf-8 -*-
# ====== טופס סטודנטים בקולאב עם עדכון ל-JSON בדרייב ======
from google.colab import drive
drive.mount('/content/drive')

import os, json
from ipywidgets import Dropdown, Text, Textarea, Button, HBox, VBox, HTML, Layout, Output
from IPython.display import display, clear_output

# ---- נתיב לקובץ בדרייב (אפשר לשנות אם שמתם בתיקייה אחרת) ----
STUDENTS_PATH = '/content/drive/MyDrive/Colab Notebooks/students.json'



def ensure_file():
  if not os.path.exists(STUDENTS_PATH):
    raise FileNotFoundError(f"⚠️ הקובץ {STUDENTS_PATH} לא נמצא בדרייב. ודא שהעלית אותו.")


def load_students():
  with open(STUDENTS_PATH, 'r', encoding='utf-8') as f:
    return json.load(f)

def save_students(data):
  with open(STUDENTS_PATH, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

ensure_file()
students = load_students()

# ---- תצוגה ועיצוב קליל ----
style_html = HTML("""
<style>
  /* רקע כללי של הכרטיס */
  .card {
    background: #1f2937;
    border-radius: 18px;
    padding: 24px 40px;
    box-shadow: 0 0 12px rgba(0,0,0,0.3);
    font-family: 'Segoe UI', sans-serif;
    color: #e5e7eb;
    max-width: 300px;          /* 👈 מגביל רוחב */
    margin: 30px auto;         /* 👈 ממרכז אופקית */
  }

  /* כותרת ראשית */
  .title {
    font-size: 24px;
    font-weight: bold;
    color: #60a5fa;
    margin-bottom: 12px;
    text-align: center;
  }

  /* טקסט עזר */
  .muted {
    color: #9ca3af;
    margin-bottom: 18px;
    font-size: 14px;
    text-align: center;
  }

  /* שדות טקסט */
  input, textarea {
    border-radius: 8px !important;
    border: 1px solid #374151 !important;
    background: #111827 !important;
    color: #f3f4f6 !important;
    padding: 8px !important;
  }

  /* כפתור */
  button[data-md-color-success] {
    background-color: #2563eb !important;
    color: #fff !important;
    font-weight: 600;
    border-radius: 8px;
    padding: 8px 18px;
  }

  /* הודעת הצלחה */
  .success { color: #22c55e; font-weight: 600; }

  /* הודעת שגיאה */
  .error { color: #f87171; font-weight: 600; }
</style>
""")



title = HTML('<div class="title">📋 טופס סטודנטים</div>')
hint = HTML('<div class="muted">בחרו סטודנט, צפו בפרטים, ועדכנו "תוכנית אהובה".</div>')

# ---- ווידג׳טים ----
options = [(f'{s["firstName"]} {s["lastName"]}  •  {s["email"]}', idx) for idx, s in enumerate(students)]
dd = Dropdown(options=options, description='סטודנט:', layout=Layout(width='100%'))

txt_first = Text(description='שם:', disabled=True, layout=Layout(width='100%'))
txt_last = Text(description='שם משפחה:', disabled=True, layout=Layout(width='100%'))
txt_email = Text(description='מייל:', disabled=True, layout=Layout(width='100%'))
ta_courses = Textarea(description='קורסים:', disabled=True, layout=Layout(width='100%', height='85px'))
txt_link = Text(description='קישור מעניין:', disabled=True, layout=Layout(width='100%'))
txt_fav = Text(description='תוכנית אהובה:', placeholder='למשל: VS Code / PyCharm / MATLAB / Netflix וכו׳', layout=Layout(width='100%'))

btn_update = Button(description='עדכון', button_style='success', layout=Layout(width='150px', height='36px'))
msg = Output()

def fill_fields(i):
  s = students[i]
  txt_first.value = s.get('firstName', '')
  txt_last.value = s.get('lastName', '')
  txt_email.value = s.get('email', '')
  ta_courses.value = ', '.join(s.get('courses', []))
  txt_link.value = s.get('interestingLink', '')
  txt_fav.value = s.get('favoriteProgram', '')

def on_change_student(change):
  if change['name'] == 'value' and change['new'] is not None:
    fill_fields(change['new'])
    with msg:
      clear_output()
      print('')

dd.observe(on_change_student, names='value')
fill_fields(dd.value)  # מילוי ראשוני

def on_update_clicked(b):
  idx = dd.value
  try:
    students[idx]['favoriteProgram'] = txt_fav.value.strip()
    save_students(students)
    with msg:
      clear_output()
      display(HTML('<span class="success">נשמר בהצלחה ב-students.json ✔</span>'))
  except Exception as e:
    with msg:
      clear_output()
      display(HTML(f'<span class="error">שגיאה בשמירה: {e}</span>'))

btn_update.on_click(on_update_clicked)

# ---- תצוגה ----
card = VBox([
  title,
  hint,
  dd,
  HTML('<div class="row">'),
  VBox([HTML('<div class="label">פרטים</div>'), txt_first, txt_last, txt_email], layout=Layout(width='100%')),
  ta_courses,
  txt_link,
  txt_fav,
  HBox([btn_update]),
  msg
], layout=Layout(width='100%'))

display(style_html, HTML('<div class="card">'), card, HTML('</div>'))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


HTML(value="\n<style>\n  /* רקע כללי של הכרטיס */\n  .card {\n    background: #1f2937;\n    border-radius: 18p…

HTML(value='<div class="card">')

HTML(value='</div>')

In [10]:
!ls /content/drive


MyDrive


In [11]:
!find /content/drive -name "students.json"


/content/drive/MyDrive/Colab Notebooks/students.json


In [19]:
# -*- coding: utf-8 -*-
# ====== טופס סטודנטים בקולאב עם עדכון ל-JSON בדרייב (Dark + צר) ======
from google.colab import drive
drive.mount('/content/drive')

import os, json
from ipywidgets import Dropdown, Text, Textarea, Button, HBox, VBox, HTML, Layout, Output, Box
from IPython.display import display, clear_output

# ---- נתיב לקובץ בדרייב (שנה אם שמור בתיקייה אחרת) ----
STUDENTS_PATH = '/content/drive/MyDrive/Colab Notebooks/students.json'

def ensure_file():
  if not os.path.exists(STUDENTS_PATH):
    raise FileNotFoundError(f"⚠️ הקובץ {STUDENTS_PATH} לא נמצא בדרייב. ודא שהעלית אותו.")

def load_students():
  with open(STUDENTS_PATH, 'r', encoding='utf-8') as f:
    return json.load(f)

def save_students(data):
  with open(STUDENTS_PATH, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

ensure_file()
students = load_students()

# ---- עיצוב כהה מודרני ----
style_html = HTML("""
<style>
  .title {
    font-size: 24px;
    font-weight: bold;
    color: #60a5fa;
    margin-bottom: 12px;
    text-align: center;
  }
  .muted {
    color: #9ca3af;
    margin-bottom: 18px;
    font-size: 14px;
    text-align: center;
  }
  input, textarea {
    border-radius: 8px !important;
    border: 1px solid #374151 !important;
    background: #111827 !important;
    color: #f3f4f6 !important;
    padding: 8px !important;
  }
  button[data-md-color-success] {
    background-color: #2563eb !important;
    color: #fff !important;
    font-weight: 600;
    border-radius: 8px;
    padding: 8px 18px;
  }
  .success { color: #22c55e; font-weight: 600; }
  .error   { color: #f87171; font-weight: 600; }
</style>
""")

title = HTML('<div class="title">📋 טופס סטודנטים</div>')
hint  = HTML('<div class="muted">בחרו סטודנט, צפו בפרטים, ועדכנו "תוכנית אהובה".</div>')

# ---- ווידג׳טים ----
options   = [(f'{s["firstName"]} {s["lastName"]}  •  {s["email"]}', idx) for idx, s in enumerate(students)]
dd        = Dropdown(options=options, description='סטודנט:', layout=Layout(width='100%'))
txt_first = Text(description='שם:', disabled=True, layout=Layout(width='100%'))
txt_last  = Text(description='שם משפחה:', disabled=True, layout=Layout(width='100%'))
txt_email = Text(description='מייל:', disabled=True, layout=Layout(width='100%'))
ta_courses= Textarea(description='קורסים:', disabled=True, layout=Layout(width='100%', height='85px'))
txt_link  = Text(description='קישור מעניין:', disabled=True, layout=Layout(width='100%'))
txt_fav   = Text(description='תוכנית אהובה:', placeholder='למשל: VS Code / PyCharm / MATLAB / Netflix וכו׳', layout=Layout(width='100%'))

btn_update= Button(description='עדכון', button_style='success', layout=Layout(width='150px', height='36px'))
msg       = Output()

def fill_fields(i):
  s = students[i]
  txt_first.value  = s.get('firstName', '')
  txt_last.value   = s.get('lastName', '')
  txt_email.value  = s.get('email', '')
  ta_courses.value = ', '.join(s.get('courses', []))
  txt_link.value   = s.get('interestingLink', '')
  txt_fav.value    = s.get('favoriteProgram', '')

def on_change_student(change):
  if change['name'] == 'value' and change['new'] is not None:
    fill_fields(change['new'])
    with msg:
      clear_output()

dd.observe(on_change_student, names='value')
fill_fields(dd.value)  # מילוי ראשוני

def on_update_clicked(b):
  idx = dd.value
  try:
    students[idx]['favoriteProgram'] = txt_fav.value.strip()
    save_students(students)
    with msg:
      clear_output()
      display(HTML('<span class="success">נשמר בהצלחה ב-students.json ✔</span>'))
  except Exception as e:
    with msg:
      clear_output()
      display(HTML(f'<span class="error">שגיאה בשמירה: {e}</span>'))

btn_update.on_click(on_update_clicked)

# ---- מבנה + צמצום רוחב אמיתי ----
card = VBox([
  title,
  hint,
  dd,
  VBox([HTML('<b>פרטים</b>'), txt_first, txt_last, txt_email], layout=Layout(width='100%')),
  ta_courses,
  txt_link,
  txt_fav,
  HBox([btn_update]),
  msg
])

# מגביל ומרכז את הכרטיס עצמו (זה החלק שאכן מצמצם את הרוחב)
card.layout = Layout(
    max_width='420px',   # ← שנה ל-360/500 לפי הטעם
    width='100%',
    margin='24px auto',
    padding='20px',
    border='1px solid #374151',
    border_radius='12px',
    background_color='#1f2937',
)

# עוטפים במיכל שמרכז בעמוד
container = Box([card])
container.layout = Layout(
    justify_content='center',
    align_items='center',
    width='100%'
)

# ---- תצוגה ----
display(style_html, container)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


HTML(value='\n<style>\n  .title {\n    font-size: 24px;\n    font-weight: bold;\n    color: #60a5fa;\n    marg…

Box(children=(VBox(children=(HTML(value='<div class="title">📋 טופס סטודנטים</div>'), HTML(value='<div class="m…